# Extract Data Frmes

In [1]:
import pandas as pd

# Load datasets
veg_df = pd.read_csv('raw_files/Vegetable-Prices-2022.csv')
fru_df = pd.read_csv('raw_files/Fruit-Prices-2022.csv')
more_df = pd.read_csv('raw_files/pp_national_average_prices_csv.csv', encoding='ISO-8859-1')
off_df = pd.read_csv('raw_files/hidden_treasures_groceries_gmm.csv')

## Check Naming Convention and Standardize the values

In [2]:
# Check column names in each dataset
print(veg_df.columns)
print(fru_df.columns)
print(more_df.columns)
print(off_df.columns)

Index(['Vegetable', 'Form', 'RetailPrice', 'RetailPriceUnit', 'Yield',
       'CupEquivalentSize', 'CupEquivalentUnit', 'CupEquivalentPrice'],
      dtype='object')
Index(['Fruit', 'Form', 'RetailPrice', 'RetailPriceUnit', 'Yield',
       'CupEquivalentSize', 'CupEquivalentUnit', 'CupEquivalentPrice'],
      dtype='object')
Index(['year', 'food_code', 'mod_code', 'food_description', 'method',
       'method_description', 'nhanes', 'price_100gm'],
      dtype='object')
Index(['Unnamed: 0', 'energy_100g', 'fat_100g', 'carbohydrates_100g',
       'sugars_100g', 'proteins_100g', 'salt_100g', 'g_sum', 'other_carbs',
       'reconstructed_energy', 'boxcox_carbohydrates_100g',
       'transformed_carbohydrates_100g', 'boxcox_fat_100g',
       'transformed_fat_100g', 'boxcox_proteins_100g',
       'transformed_proteins_100g', 'boxcox_sugars_100g',
       'transformed_sugars_100g', 'boxcox_other_carbs',
       'transformed_other_carbs', 'boxcox_salt_100g', 'transformed_salt_100g',
       'boxco

In [3]:
#Rename columns
veg_df.rename(columns={'Vegetable': 'product'}, inplace=True)
fru_df.rename(columns={'Fruit': 'product'}, inplace=True)
more_df.rename(columns={'food_description': 'product'}, inplace=True)

#keep only relevant columns
veg_df = veg_df[['product', 'RetailPrice']]
fru_df = fru_df[['product', 'RetailPrice']]
more_df = more_df[['product', 'price_100gm']]


In [4]:
#more_df is from 2012 and in different units.  doubling the price for inflation and converting from 100g to price per lb

inflation_factor = 2
grams_per_pound = 453.592
price_per_100g_to_lb = grams_per_pound / 100

# Adjusting for inflation and converting units
# Use .loc to avoid the SettingWithCopyWarning
more_df.loc[:, 'RetailPrice'] = more_df['price_100gm'] * inflation_factor * price_per_100g_to_lb
more_df = more_df.drop(columns=['price_100gm'])

## Inner Merge on Food Product

In [5]:
#Merge the vegetables on 'product' and fruits on 'product' using an inner join
vegnut_df = pd.merge(veg_df, off_df, on='product', how='inner')
fruitnut_df = pd.merge(fru_df, off_df, on='product', how='inner')
morenut_df = pd.merge(more_df, off_df, on='product', how='inner')

In [6]:
#drop duplicates
vegnut_df.drop_duplicates(subset='product', inplace=True)
fruitnut_df.drop_duplicates(subset='product', inplace=True)
morenut_df.drop_duplicates(subset='product', inplace=True)

In [7]:
vegnut_df.to_csv('merged_files/veg.csv', index=False)
fruitnut_df.to_csv('merged_files/fruit.csv', index=False)
morenut_df.to_csv('merged_files/more.csv', index=False)

# Stack Data Frames

In [8]:
working_df = pd.concat([vegnut_df, fruitnut_df, morenut_df], ignore_index=True)


In [9]:


# Dropping unnecessary columns such as 'Unnamed: 0'
df_cleaned = working_df.drop(columns=['Unnamed: 0'])

# Step 2: Check for missing values
missing_values = df_cleaned.isnull().sum()

# Step 3: Drop rows with missing target values (if any)
df_cleaned = df_cleaned.dropna(subset=['RetailPrice'])


#df_cleaned.to_csv('working.csv', index=False)